# 국경일, 공휴일, 기념일, 잡절 api 링크 정의

In [106]:
import pandas as pd
import numpy as np
holiday = "http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/getHoliDeInfo?serviceKey="
restday = "http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/getRestDeInfo?serviceKey="
anniversary = "http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/getAnniversaryInfo?serviceKey="
division_24 = "http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/getSundryDayInfo?serviceKey="
sundri = "http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/get24DivisionsInfo?serviceKey="
apikey = "VQuidVYunrTqlOTjmoW0ahJXzyaqCVYlE7tmwYbpmrCjT5kPmsXeMxgmUHU0y9aTgps1Tg%2B6uTLpswjX%2FZ89QA%3D%3D"
year = "&solYear=2022&numOfRows=1000"

api_dict = {"datekind": ["holiday","restday","anniversary","division_24","sundri"], 
            "api_url": [holiday, restday, anniversary, division_24, sundri]}
api_df = pd.DataFrame(api_dict)

# bs 사용해서 df_cal에다가 캘린더 저장

In [115]:
import requests
from bs4 import BeautifulSoup as bs
xml_columns = ['datekind','datename','isholiday','locdate','seq']
df_cal= pd.DataFrame()
for kind in range(0,len(api_df)):
    page = requests.get(api_df.iloc[kind,1] + apikey + year)
    soup = bs(page.text, "lxml")
    
    elements = soup.select('div.esg-entry-content a > span')
    
    df = pd.DataFrame([soup.find_all(x) for x in xml_columns]).transpose()
    df.columns = xml_columns
    df_cal = df_cal.append(df)
df_cal.reset_index(inplace=True, drop=True)
df_cal

,datekind,datename,isholiday,locdate,seq
0,[01],[1월1일],[Y],[20220101],[1]
1,[01],[설날],[Y],[20220131],[1]
2,[01],[설날],[Y],[20220201],[1]
3,[01],[설날],[Y],[20220202],[1]
4,[01],[삼일절],[Y],[20220301],[1]
...,...,...,...,...,...
131,[03],[상강],[N],[20221023],[1]
132,[03],[입동],[N],[20221107],[1]
133,[03],[소설],[N],[20221122],[1]
134,[03],[대설],[N],[20221207],[1]


# bs_object 형식을 object 형식으로 변환

In [116]:
for C in range(0,df_cal.shape[1]):
    df_cal.iloc[:,C] = df_cal.iloc[:,C].map(lambda x: x.get_text())

df_cal.drop_duplicates(inplace=True)
df_cal.reset_index(inplace=True, drop=True)
df_cal

,datekind,datename,isholiday,locdate,seq
0,01,1월1일,Y,20220101,1
1,01,설날,Y,20220131,1
2,01,설날,Y,20220201,1
3,01,설날,Y,20220202,1
4,01,삼일절,Y,20220301,1
...,...,...,...,...,...
112,03,상강,N,20221023,1
113,03,입동,N,20221107,1
114,03,소설,N,20221122,1
115,03,대설,N,20221207,1


# csv 출력

In [118]:
df_cal.to_csv('df_cal.csv', sep=',', na_rep='NaN')